In [1]:
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
stock = pd.read_csv("C:\\AIdata\\Online Retail.CSV")

In [4]:
stock

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50,4.15,12680.0,France


In [6]:
stock.columns = ['InvoiceNo','StockCode', 'Description', 'Quantity','InvoiceDate','UnitPrice','CustomerID','Country']

In [7]:
stock['Description'].value_counts()

Description
WHITE HANGING HEART T-LIGHT HOLDER     2369
REGENCY CAKESTAND 3 TIER               2200
JUMBO BAG RED RETROSPOT                2159
PARTY BUNTING                          1727
LUNCH BAG RED RETROSPOT                1638
                                       ... 
Missing                                   1
historic computer difference?....se       1
DUSTY PINK CHRISTMAS TREE 30CM            1
WRAP BLUE RUSSIAN FOLKART                 1
PINK BERTIE MOBILE PHONE CHARM            1
Name: count, Length: 4223, dtype: int64

In [8]:
stock_AJ = stock[stock['StockCode']=='85123A']

In [9]:
stock_AJ = stock_AJ [['InvoiceNo','Quantity','UnitPrice','CustomerID']]

In [10]:
stock_AJ

,InvoiceNo,Quantity,UnitPrice,CustomerID
0,536365,6,2.55,17850.0
49,536373,6,2.55,17850.0
66,536375,6,2.55,17850.0
220,536390,64,2.55,17511.0
262,536394,32,2.55,13408.0
...,...,...,...,...
539342,581439,1,5.79,NaN
539979,581452,32,2.55,17675.0
540217,581472,6,2.95,15796.0
540837,581492,3,5.79,NaN


In [11]:
stock_AJ['InvoiceNo'].value_counts()

InvoiceNo
560271    3
551057    3
547670    3
568301    3
571461    3
         ..
548714    1
548711    1
548708    1
548699    1
581538    1
Name: count, Length: 2246, dtype: int64

In [15]:
def MinMaxScaler(data):
    """최솟값과 최댓값을 이용하여 0 ~ 1 값으로 변환"""
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    return numerator / (denominator + 1e-7)

In [17]:
dfx = stock_AJ[['Quantity','UnitPrice','CustomerID']]
dfx = MinMaxScaler(dfx)
dfy = dfx[['UnitPrice']]
dfx = dfx[['Quantity','CustomerID']]
dfx

,Quantity,CustomerID
0,0.326476,0.926772
49,0.326476,0.926772
66,0.326476,0.926772
220,0.336256,0.869440
262,0.330860,0.175545
...,...,...
539342,0.325632,NaN
539979,0.330860,0.897176
540217,0.326476,0.579401
540837,0.325970,NaN


In [18]:
dfx = dfx.dropna()

In [19]:
dfx

,Quantity,CustomerID
0,0.326476,0.926772
49,0.326476,0.926772
66,0.326476,0.926772
220,0.336256,0.869440
262,0.330860,0.175545
...,...,...
538708,0.326138,0.345848
539083,0.330860,0.241502
539979,0.330860,0.897176
540217,0.326476,0.579401


In [20]:
dfx.describe()

,Quantity,CustomerID
count,2077.000000,2077.000000
mean,0.328241,0.539540
std,0.014724,0.273233
min,0.000000,0.000000
25%,0.325970,0.316083
50%,0.326476,0.545916
75%,0.327487,0.771351
max,0.650927,1.000000


In [21]:
X = dfx.values.tolist()
y = dfy.values.tolist()

In [28]:
# 데이터 수집 코드
window_size = 8
data_X = []
data_y = []
for i in range(len(y) - window_size):
    _X = X[i : i + window_size] 
    _y = y[i + window_size]     
    data_X.append(_X)
    data_y.append(_y)
    if i < 10 :
        print(_X, "->", _y)

[[0.3264755480552028, 0.9267715203631529], [0.3264755480552028, 0.9267715203631529], [0.3264755480552028, 0.9267715203631529], [0.3362563237717326, 0.869440216457476], [0.3308600337212334, 0.17554540841915195], [0.3264755480552028, 0.9267715203631529], [0.3261382799270466, 0.5905631658956442], [0.326812816183359, 0.9267715203631529]] -> [0.4991539678654151]
[[0.3264755480552028, 0.9267715203631529], [0.3264755480552028, 0.9267715203631529], [0.3362563237717326, 0.869440216457476], [0.3308600337212334, 0.17554540841915195], [0.3264755480552028, 0.9267715203631529], [0.3261382799270466, 0.5905631658956442], [0.326812816183359, 0.9267715203631529], [0.3264755480552028, 0.7072552003939243]] -> [0.4991539678654151]
[[0.3264755480552028, 0.9267715203631529], [0.3362563237717326, 0.869440216457476], [0.3308600337212334, 0.17554540841915195], [0.3264755480552028, 0.9267715203631529], [0.3261382799270466, 0.5905631658956442], [0.326812816183359, 0.9267715203631529], [0.3264755480552028, 0.70725

In [29]:
print('전체 데이터의 크기 :', len(data_X), len(data_y))

전체 데이터의 크기 : 2305 2305


In [47]:
train_size = int(len(data_y) * 0.7)
train_X = np.array(data_X[0 : train_size], dtype=object)
train_y = np.array(data_y[0 : train_size])

test_size = len(data_y) - train_size
test_X = np.array(data_X[train_size : len(data_X)], dtype=object)
test_y = np.array(data_y[train_size : len(data_y)])

print('훈련 데이터의 크기 :', train_X.shape, train_y.shape)
print('테스트 데이터의 크기 :', test_X.shape, test_y.shape)

훈련 데이터의 크기 : (1613, 8, 2) (1613, 1)
테스트 데이터의 크기 : (692,) (692, 1)


In [31]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, SimpleRNN

In [32]:
model = Sequential()
model.add(LSTM(units=20, activation='relu', return_sequences=True, input_shape=(8, 2)))
model.add(Dropout(0.1))
model.add(SimpleRNN(units=20, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1))
model.summary()

# 모델 초기화
model = Sequential()

# 첫 번째 RNN 레이어와 Dropout 레이어 추가
model.add(SimpleRNN(20, activation='relu', input_shape=(8,2), return_sequences=True))
model.add(Dropout(0.5))

# 두 번째 RNN 레이어와 Dropout 레이어 추가
model.add(SimpleRNN(20, activation='relu'))
model.add(Dropout(0.3))

# 최종 출력을 위한 Dense 레이어 추가
model.add(Dense(1))

# 모델 요약 출력
model.summary()

C:\Users\kgg07\anaconda3\envs\learning3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 8, 20)               │           1,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 8, 20)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 20)                  │             820 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,681 (10.47 KB)

 Trainable params: 2,681 (10.47 KB)

...

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)             │ (None, 8, 20)               │             460 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 8, 20)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ (None, 20)                  │             820 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,301 (5.08 KB)

 Trainable params: 1,301 (5.08 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
model = Sequential()
model.add(LSTM(units=20, activation='relu', return_sequences=True, input_shape=(8, 2)))
model.add(Dropout(0.1))
model.add(LSTM(units=20, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(units=1))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 8, 20)               │           1,840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 8, 20)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 20)                  │           3,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 20)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,141 (20.08 KB)

 Trainable params: 5,141 (20.08 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(train_X, train_y, epochs=70, batch_size=30)
pred_y = model.predict(test_X)

Epoch 1/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.0176
Epoch 2/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0169
Epoch 3/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0185
Epoch 4/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0182
Epoch 5/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0178
Epoch 6/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0195
Epoch 7/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0181
Epoch 8/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0187
Epoch 9/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0159
Epoch 10/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0181
Epoch 11/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0172
Epoch 12/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0185
Epoch 13/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0187
Epoch 14/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0189
Epoch 15/70
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0177
Epoch 16/70
54/54 ━

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [67]:
print("test_X:", test_X[:5])  # 첫 5개 샘플 출력
print("test_y:", test_y[:5])  # 첫 5개 샘플 출력

test_X: [[[list([[0.32748735243967136, 0.9611026551503281], [0.3259696458629685, 0.1510231692854554], [0.32580101179889037, 0.23575173346464146], [0.32580101179889037, 0.7371892440260919], [0.3308600337212334, 0.8971757145121397], [0.3261382799270466, 0.45053272449770787], [0.3264755480552028, 0.514290546245319], [0.3264755480552028, 0.6822256045884961]])]]

 [[list([[0.3259696458629685, 0.1510231692854554], [0.32580101179889037, 0.23575173346464146], [0.32580101179889037, 0.7371892440260919], [0.3308600337212334, 0.8971757145121397], [0.3261382799270466, 0.45053272449770787], [0.3264755480552028, 0.514290546245319], [0.3264755480552028, 0.6822256045884961], [0.3308600337212334, 0.07982411635244674]])]]

 [[list([[0.32580101179889037, 0.23575173346464146], [0.32580101179889037, 0.7371892440260919], [0.3308600337212334, 0.8971757145121397], [0.3261382799270466, 0.45053272449770787], [0.3264755480552028, 0.514290546245319], [0.3264755480552028, 0.6822256045884961], [0.3308600337212334, 0

In [73]:
test_X = np.array([np.array(item[0]) for item in test_X], dtype=np.float32)

# shape이 (샘플 수, 시퀀스 길이, 특성 수)로 맞추기
# 예: 현재 샘플 수는 5이고, 각 샘플의 시퀀스 길이는 8, 특성 수는 2라면
# test_X의 최종 shape은 (5, 8, 2)이어야 함
test_X = test_X.reshape(test_X.shape[0], test_X.shape[1], -1)  # 특성 수는 자동으로 조정

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (692,) + inhomogeneous part.